In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import mayfly as mf
import scipy.signal
import scipy.stats
import scipy.interpolate
from scipy import integrate
from pathlib import Path

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'datasets/data')
#SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""
    

In [ ]:
os.listdir(os.path.join(DATAPATH))

# load data

In [ ]:
# summed data
data_path = Path.home()/'group'/'project'/'datasets'/'data'/'bf'
data_list = list(data_path.iterdir())
for i in data_list: print(i)

data = data_list[1]
data = np.load(data)

In [ ]:
signal = data[5690, :]

var = 1.38e-23 * 10 * 50 * 200e6 * 60 * np.sqrt(60)
noise = np.random.multivariate_normal(np.zeros(2), np.eye(2) * var / 2, 8192)
noise = noise[:, 0] + 1j * noise[:, 1]



In [ ]:
noisy_spec = abs(np.fft.fft(signal + noise) / 8192) ** 2

plt.plot(noisy_spec)
print(var / 8192, np.mean(noisy_spec), 10 * np.log10(np.max(noisy_spec) / np.mean(noisy_spec)))

In [ ]:
x = np.linspace(0,10e-12, 201)

pdf = scipy.stats.genextreme.pdf(x, -0.5, loc=3e-12, scale = 1e-12)

In [ ]:
plt.plot(x, pdf)

In [ ]:
n_samples = 5000
max_value = np.zeros(n_samples)
for i in range(n_samples):
    noise = np.random.multivariate_normal(np.zeros(2), np.eye(2) * var / 2, 8192)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    noisy_spec = abs(np.fft.fft(signal + noise) / 8192) ** 2
    
    max_value[i] = noisy_spec.max()
    
    

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))

ax = fig.add_subplot(1,1,1)
hist = ax.hist(max_value, 50, density=True)

x = np.linspace(0,4e-12, 201)
pdf = scipy.stats.rice.pdf(x, 7.8e-13, loc=7.8e-13 / 2, scale = 7.8e-13 / 2)

print(np.max(abs(np.fft.fft(signal ) / 8192) ** 2))

ax.plot(x, pdf)

In [ ]:
n_samples = 5000
max_value_noise = np.zeros(n_samples)
for i in range(n_samples):
    noise = np.random.multivariate_normal(np.zeros(2), np.eye(2) * var / 2, 8192)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    noisy_spec = abs(np.fft.fft(signal + noise) / 8192) ** 2
    
    max_value_noise[i] = noisy_spec.max()
    
    

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))

ax = fig.add_subplot(1,1,1)
hist = ax.hist(max_value_noise, 50, density=True)

x = np.linspace(0,4e-12, 201)
pdf = scipy.stats.rayleigh.pdf(x, loc=1e-12/1.8, scale=1.06e-12/2.5)

#print(np.max(abs(np.fft.fft(signal ) / 8192) ** 2))
print(np.mean(max_value_noise))
ax.plot(x, pdf)

In [ ]:
data_fft = np.fft.fft(data, axis=-1) / data.shape[-1]

In [ ]:
var_fft = 1.38e-23 * 10 * 50 * 200e6 * 60 / 8192

In [ ]:
n_samples = 10
max_value_noise = np.zeros((n_samples, data_fft.shape[0]))
for i in range(n_samples):
    noise = np.random.multivariate_normal(np.zeros(2), np.eye(2) * var_fft / 2, data_fft.size)
    noise = (noise[:, 0] + 1j * noise[:, 1]).reshape(data_fft.shape)
    
    noisy_spec = abs(data_fft + noise) ** 2
    
    max_value_noise[i, :] = noisy_spec.max(axis = -1)
    
    
    print(i + 1)

# down-select data

In [ ]:
h5file = h5py.File(os.path.join(DATAPATH, '211116_grad_b_est.h5'), 'r')
data = mf.data.MFDataset(os.path.join(DATAPATH, '211116_grad_b_est.h5'))
metadata = pd.DataFrame(data.metadata)

In [ ]:
radial_positions = np.unique(metadata['x_min'][:])
pitch_angles = np.unique(metadata['theta_min'][:])

print(radial_positions[0:51], pitch_angles[20:101])

In [ ]:
radial_positions = np.unique(metadata['x_min'][:])
pitch_angles = np.unique(metadata['theta_min'][:])

radial_positions = radial_positions[np.arange(10, 41, 2)]
pitch_angles = pitch_angles[np.arange(20, 101, 2)]


#print(radial_positions[np.arange(0, 51, 3)], radial_positions[np.arange(0, 51, 3)].size)
#print(pitch_angles[np.arange(0, 121, 3)], pitch_angles[np.arange(0, 121, 3)].size)
rad_grid, angle_grid = np.meshgrid(radial_positions, pitch_angles)
    
sorted_signals = np.zeros((rad_grid.size, data_fft.shape[-1]), dtype=np.complex64)

for i, pair in enumerate(zip(rad_grid.flatten(), angle_grid.flatten())):
    #print(np.argwhere(h5file['meta']['x_min'][:].array == pair[0]))
    #print(np.argwhere(h5file['meta']['theta_min'][:] == pair[1]))
    try:
        index = np.argwhere(np.logical_and(h5file['meta']['x_min'][:] == pair[0], h5file['meta']['theta_min'][:] == pair[1])).squeeze()
        #print(index)
        sorted_signals[i, :] = data_fft[index, :]
    except BaseException as err:
        #print(err)
        pass
    
print(sorted_signals.shape)

In [ ]:
n_samples = 1000
max_value_noise = np.zeros((n_samples, sorted_signals.shape[0]))
for i in range(n_samples):
    noise = np.random.multivariate_normal(np.zeros(2), np.eye(2) * var_fft / 2, sorted_signals.size)
    noise = (noise[:, 0] + 1j * noise[:, 1]).reshape(sorted_signals.shape)
    
    noisy_spec = abs(sorted_signals + noise) ** 2
    
    max_value_noise[i, :] = noisy_spec.max(axis = -1)
    
    
    print(i + 1)

In [ ]:
max_value_noise.shape

In [ ]:
hist = np.histogram(max_value_noise[:, 100], bins=50, density=True)

In [ ]:
plt.plot(hist[1][1:], hist[0],)

In [ ]:
var_fft = 1.38e-23 * 10 * 50 * 200e6 * 60 / 8192

n_samples = 1000000
max_value_noise_only = np.zeros((n_samples))
for i in range(n_samples):
    noise = np.random.multivariate_normal(np.zeros(2), np.eye(2) * var_fft / 2, 8192)
    noise = (noise[:, 0] + 1j * noise[:, 1])
    
    noisy_spec = abs(noise) ** 2
    #print(noisy_spec.shape)
    
    max_value_noise_only[i] = noisy_spec.max()
    
    
#print(i + 1)

In [ ]:
np.save(Path.home()/'group'/'project'/'results'/'beamforming'/'pdf'/'220304_10K_noise_only_Vsquared_power_spectra_max.npy', max_value_noise_only)

In [ ]:
#hist = np.histogram(max_value_noise[:, 6], bins=np.linspace(0, 6e-12, 128), density=True)
#plt.plot(hist[1][1:], hist[0],)
hist = np.histogram(max_value_noise_only, bins=np.linspace(0, 0.2e-12, 128), density=True)
plt.plot(hist[1][1:], hist[0],)

# estimate pdf

In [ ]:
pdf_estimates = np.zeros((max_value_noise.shape[-1], 2, 1024-1))

for i in range(max_value_noise.shape[-1]):
    
    hist = np.histogram(max_value_noise[:, i], bins=np.linspace(0, 6e-12, 1024), density=True)
    
    pdf_estimates[i, 0, :] = hist[0]
    pdf_estimates[i, 1, :] = hist[1][1:]

# estimate cdf

In [ ]:
plt.plot(pdf_estimates[0,1,:], pdf_estimates[0,0,:])

print(integrate.simpson(pdf_estimates[0,0,:], x=pdf_estimates[0, 1, :]))

In [ ]:
cdf_estimates = np.zeros((pdf_estimates.shape))
for n in range(pdf_estimates.shape[-1]):
    cdf_estimates[:, 0, n] = integrate.trapezoid(pdf_estimates[:,0,0:(n + 1)], x=pdf_estimates[:, 1, 0:(n+1)], axis=-1)
cdf_estimates[:, 1, :] = pdf_estimates[:, 1, :]

In [ ]:
plt.plot(cdf_estimates[600,1,:], 1-cdf_estimates[600, 0, :])

# estimate noise only cdf

In [ ]:
pdf_noise = np.zeros((2, 1024-1))
cdf_noise = np.zeros(pdf_noise.shape)

hist = np.histogram(max_value_noise_only, bins=np.linspace(0, 6e-12, 1024), density=True)
    
pdf_noise[0, :] = hist[0]
pdf_noise[1, :] = hist[1][1:]

for n in range(pdf_noise.shape[-1]):
    cdf_noise[0, n] = integrate.trapezoid(pdf_noise[0, 0:n+1], x=pdf_noise[1, 0:n+1],)
cdf_noise[1, :] = pdf_noise[1, :]

plt.plot(cdf_estimates[60,1,:], 1-cdf_estimates[60, 0, :])
plt.plot(cdf_noise[1, :], 1-cdf_noise[0, :])


In [ ]:
plt.plot(1-cdf_noise[0, :], 1-cdf_estimates[500, 0, :])

plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10), color='grey', linestyle='--')

In [ ]:
plt.plot(1-cdf_noise[0, :], 1-np.mean(cdf_estimates[:, 0, :], axis=0), '-')

plt.plot(np.linspace(0, 1, 401), np.linspace(0, 1, 401), color='grey', linestyle='--')

plt.xscale('log')
plt.xlim(1e-8, 1)

# save the roc curve data

In [ ]:
save_path = Path.home()/'group'/'project'/'results'/'beamforming'/'roc_curve'/'220223_time_dep_beamforming_roc_curve_estimate_restricted_params.npz'
np.savez(save_path, fpr=1-cdf_noise[0, :], tpr=1-np.mean(cdf_estimates[:, 0, :], axis=0))